In [3]:
from dash import Dash, dcc, html, Output, Input, State
import dash_bootstrap_components as dbc
import pandas as pd
import threading
import webbrowser

# Initialize Dash app with suppress_callback_exceptions
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP], suppress_callback_exceptions=True)
app.title = "Guía Turística de Valencia"

# Initialize DataFrame with example users
user_info = pd.DataFrame({
    'user_id': range(1, 11),
    'Nombre': [f'User_{i}' for i in range(1, 11)],
    'Edad': [25 + i for i in range(10)],
    'Sexo': ['F' if i % 2 == 0 else 'M' for i in range(10)],
    'Ocupación': [str((i % 11) + 1) for i in range(10)],
    'Hijos': [1 if i % 3 == 0 else 0 for i in range(10)],
    'Edad_hijo_menor': [5 if i % 3 == 0 else 0 for i in range(10)],
    'Edad_hijo_mayor': [8 if i % 3 == 0 else 0 for i in range(10)]
})

# Options for "Ocupación"
ocupacion_options = [
    {"label": "Fuerzas armadas", "value": "1"},
    {"label": "Dirección de las empresas y de las administraciones públicas", "value": "2"},
    {"label": "Técnicos y profesionales científicos e intelectuales", "value": "3"},
    {"label": "Técnicos y profesionales de apoyo", "value": "4"},
    {"label": "Empleados de tipo administrativo", "value": "5"},
    {"label": "Trabajadores de los servicios de restauración, personales, protección y vendedores de los comercios", "value": "6"},
    {"label": "Trabajadores cualificados en la agricultura y en la pesca", "value": "7"},
    {"label": "Artesanos y trabajadores cualificados de industrias manufactureras, construcción, y minería, excepto operadores de instalaciones y maquinaria", "value": "8"},
    {"label": "Operadores de instalaciones y maquinaria, y montadores", "value": "9"},
    {"label": "Trabajadores no cualificados", "value": "10"},
    {"label": "Inactivo o desocupado", "value": "11"}
]

# App Layout
app.layout = dbc.Container(
    [
        dbc.Row(
            dbc.Col(
                html.H1("Hola, bienvenidos a tu guía turística de Valencia!", 
                        className="text-center text-white my-4", 
                        style={"background-color": "#4CAF50", "padding": "10px", "border-radius": "10px"}),
                width=12
            )
        ),
        
        dbc.Row(
            dbc.Col(
                [
                    dbc.Input(id="user_id", type="text", placeholder="Introduce tu ID de usuario", className="mb-2"),
                    dbc.Button("Login", id="login_button", color="primary", className="me-2"),
                    dbc.Button("Sign Up", id="signup_button", color="success")
                ],
                width=6, className="offset-md-3 text-center"
            )
        ),

        html.Div(id="signup_form", style={"marginTop": "30px"}),
        html.Div(id="confirmation_message")  # Added this to display confirmation messages
    ],
    fluid=True,
    style={"backgroundColor": "#E0F8E0", "height": "100vh"}  
)

# Callback to display the signup form
@app.callback(
    Output("signup_form", "children"),
    Input("signup_button", "n_clicks"),
    prevent_initial_call=True
)
def display_signup_form(n_clicks):
    return dbc.Card(
        dbc.CardBody([
            html.H4("Registro de Usuario", className="text-center"),
            
            # Edad input
            dbc.Input(id="edad", type="number", placeholder="Edad", className="mb-3"),

            # Sexo RadioItems
            dbc.Row([
                dbc.Label("Sexo:", width=2),
                dbc.Col(
                    dbc.RadioItems(
                        options=[{"label": "Femenino", "value": "F"}, {"label": "Masculino", "value": "M"}],
                        value=None,
                        id="sexo",
                        inline=True
                    ),
                    width=10
                )
            ], className="mb-3"),

            # Ocupación Dropdown (fixed dropdown behavior)
            dbc.Row([
                dbc.Label("Ocupación:", width=2),
                dbc.Col(
                    dcc.Dropdown(
                        id="ocupacion",
                        options=ocupacion_options,
                        placeholder="Selecciona tu ocupación",
                        style={'width': '100%'}
                    ),
                    width=10
                )
            ], className="mb-3"),

            # ¿Tienes hijos? Checklist
            dbc.Row([
                dbc.Label("¿Tienes hijos?", width=2),
                dbc.Col(
                    dbc.Checklist(
                        options=[{"label": "Sí", "value": 1}],
                        value=[],
                        id="hijos",
                        inline=True
                    ),
                    width=10
                )
            ], className="mb-3"),

            # Edad hijos (conditionally displayed)
            html.Div(id="hijos_edades"),

            # Submit Button
            dbc.Button("Enviar", id="submit_button", color="primary", className="w-100 mt-4")
        ]),
        style={"maxWidth": "800px", "margin": "auto", "marginTop": "20px", "backgroundColor": "#f8f9fa", "padding": "20px", "borderRadius": "10px"}
    )


# Callback to show/hide edad_hijo_menor and edad_hijo_mayor based on "¿Tienes hijos?"
@app.callback(
    Output("hijos_edades", "children"),
    Input("hijos", "value")
)
def toggle_hijos_fields(hijos):
    if 1 in hijos:
        return html.Div([
            dbc.Input(id="edad_hijo_menor", type="number", placeholder="Edad hijo menor", className="mb-2"),
            dbc.Input(id="edad_hijo_mayor", type="number", placeholder="Edad hijo mayor", className="mb-2")
        ])
    return None

@app.callback(
    Output("confirmation_message", "children"),
    Input("submit_button", "n_clicks"),
    State("edad", "value"),
    State("sexo", "value"),
    State("ocupacion", "value"),
    State("hijos", "value"),
    State("edad_hijo_menor", "value"),
    State("edad_hijo_mayor", "value"),
    prevent_initial_call=True
)
def submit_user(n_clicks, edad, sexo, ocupacion, hijos, edad_hijo_menor, edad_hijo_mayor):
    global user_info

    # Validation
    if edad is None or sexo is None or ocupacion is None:
        return dbc.Alert("Por favor, completa todos los campos obligatorios.", color="danger")

    # Auto increment user_id
    user_id = user_info['user_id'].max() + 1 if not user_info.empty else 1
    nombre = f'User_{user_id}'
    
    # Hijos
    tiene_hijos = 1 if hijos and 1 in hijos else 0
    edad_hijo_menor = edad_hijo_menor if tiene_hijos else 0
    edad_hijo_mayor = edad_hijo_mayor if tiene_hijos else 0

    # Nuevo usuario
    new_user = {
        'user_id': user_id,
        'Nombre': nombre,
        'Edad': edad,
        'Sexo': sexo,
        'Ocupación': ocupacion,
        'Hijos': tiene_hijos,
        'Edad_hijo_menor': edad_hijo_menor,
        'Edad_hijo_mayor': edad_hijo_mayor
    }
    
    # Add new user to the DataFrame
    user_info = pd.concat([user_info, pd.DataFrame([new_user])], ignore_index=True)
    
    # Print the updated DataFrame to verify (for debugging)
    print(user_info.tail())
    
    # Return success message
    return dbc.Alert(
        f"✅ Usuario {nombre} registrado exitosamente. ¡Gracias!",
        color="success"
    )


# Function to open browser automatically
def open_browser():
    webbrowser.open_new("http://127.0.0.1:8050/")

# Run the Dash app in a separate thread
if __name__ == '__main__':
    threading.Timer(1, open_browser).start()  # Open the app in the browser
    app.run_server(debug=False, port=8050)

    user_id   Nombre  Edad Sexo Ocupación  Hijos  Edad_hijo_menor  \
6         7   User_7    31    F         7      1                5   
7         8   User_8    32    M         8      0                0   
8         9   User_9    33    F         9      0                0   
9        10  User_10    34    M        10      1                5   
10       11  User_11    30    F         3      1                2   

    Edad_hijo_mayor  
6                 8  
7                 0  
8                 0  
9                 8  
10                9  


In [4]:
user_info

,user_id,Nombre,Edad,Sexo,Ocupación,Hijos,Edad_hijo_menor,Edad_hijo_mayor
0,1,User_1,25,F,1,1,5,8
1,2,User_2,26,M,2,0,0,0
2,3,User_3,27,F,3,0,0,0
3,4,User_4,28,M,4,1,5,8
4,5,User_5,29,F,5,0,0,0
5,6,User_6,30,M,6,0,0,0
6,7,User_7,31,F,7,1,5,8
7,8,User_8,32,M,8,0,0,0
8,9,User_9,33,F,9,0,0,0
9,10,User_10,34,M,10,1,5,8
